# SkillFactory
## Домашнее задание 3. Оптимизация KNN
https://www.kaggle.com/c/sfml-4-classification

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})

pd.set_option('expand_frame_repr', True)
#display.latex.multirow
pd.set_option('multirow', True)
pd.set_option('max_colwidth', 150)
#pd.set_option('precision',2)
#pd.set_eng_float_format(accuracy=3, use_eng_prefix=False)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Грузим данные

Считаем описание данных

In [2]:
df_train_origin = pd.read_csv('data/train.csv', sep=',', encoding='utf8')
df_test_origin = pd.read_csv('data/test.csv', sep=',', encoding='utf8')

In [3]:
df_dictionary = {
	"_id": 'идентификатор человека',
	"age": 'возраст',
	"job": 'тип работы',
	"marital": 'семейное положение',
	"education": 'уровень образования',
	"default": 'есть ли дефолт по кредиту',
	"housing": 'есть ли кредит на жилье',
	"loan": 'есть ли персональный кредит',
	"contact": 'контактный тип связи',
	"month": 'последний контактный месяц года',
	"day_of_week": 'последний контактный день недели',
	"duration": 'длительность последнего контакта в секундах',
	"campaign": 'количество контактов',
	"pdays": 'количество дней прошедших последнего контакта',
	"previous": 'количество контактов',
	"poutcome": 'результат предыдущей маркетинговой кампании',
	"emp.var.rate": 'изменение уровня занятости',
	"cons.price.idx": 'индекс потребительских цен',
	"cons.conf.idx": 'индекс доверия потребителя',
	"euribor3m": 'курс Euribor',
	"nr.employed": 'количество сотрудников',
	"target": 'целевая переменная',
}

In [4]:
df_train_origin.shape

(24712, 22)

In [5]:
df_test_origin.shape

(16476, 21)

In [6]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from IPython.display import display, HTML
from pandas.io.formats.style import Styler

#подготовим словарь названий
df_dict = df_dictionary
    
#выведем инфу (аля info + describe + доп значения)
'''
Опиание полей: 
название, тип, семантика поля, количество значений, количество пропусков, количество уникальных значений,  
    для целочисленных: медиана, среднее,
    для измерений: значения
наиболее популярные значения (значение+кол-во)
'''
def custom_describe(dataframe, dictionary, column):
    temp_data = {
        'Тип': [dataframe[column].dtype],
        'Значение': [dictionary[column] if column in dictionary else ''],
        
        'Количество значений': [dataframe[column].count()],
        'Количество пропусков': [len(dataframe[column]) - dataframe[column].count()],
        'Количество уникальных значений': [dataframe[column].nunique()],
    }    
    
    if (is_numeric_dtype(dataframe[column])):
        temp_data['Медиана'] = [dataframe[column].median()]
        temp_data['Среднее'] = [dataframe[column].mean()]
    else:
        temp_data['Медиана'] = [np.nan]
        temp_data['Среднее'] = [np.nan]
        
    if (is_string_dtype(dataframe[column])):
        temp_data['Значения'] = [dataframe[column].unique()]
    else:
        temp_data['Значения'] = [np.nan]
    
    #ТОП значений
    most_freq_values = dataframe[column].value_counts().nlargest(10)
       
        
    
    #рисуем
    new_df = pd.DataFrame(data=temp_data) 
    new_df = new_df.rename(index={new_df.index[0]: column})
    
    new_df_top = pd.DataFrame(data=most_freq_values) 
    
    display(new_df)
    display(new_df_top.T)
    
    '''
    print(column,dataframe[column].dtype,dictionary[column] if column in dictionary else '')
    print('Количество значений: ', dataframe[column].count())
    print('Количество пропусков: ', len(dataframe[column]) - dataframe[column].count())
    print('Количество уникальных значений: ', dataframe[column].nunique())
    
    if (is_numeric_dtype(dataframe[column])):
        print('Медиана: ', dataframe[column].median())
        print('Среднее: ', dataframe[column].mean())
        
    if (is_string_dtype(dataframe[column])):
        print('Значения: ', dataframe[column].unique())
    
    print('\nНаиболее встречающиеся значения:')
    print(dataframe[column].value_counts().nlargest(10))
    '''


In [7]:
for i in df_train_origin.columns: # перебираем все столбцы
    custom_describe(df_train_origin, df_dict, i)
    print('\n' + '-'*50)

,Тип,Значение,Количество значений,Количество пропусков,Количество уникальных значений,Медиана,Среднее,Значения
_id,object,идентификатор человека,24712,0,24712,nan,nan,"[df7489733b004bbe40d3d37b34f82419, 905a0b9a5456ee962223033473666be3, 02d69c07ec6d68e31d641edd45ce61cd, e492fbe0cccaf67cdb5c0944dbc8a167, ac2dc6f58..."


,e62da29ac3a82185101eb38cb426322a,1fd87f5668cbbe66e52689551b3c9dbf,8fe949b5c5b67214d44ed4272906dc5b,dd0ac2b2a8ec33c365f00f646be65b06,f8cd469a2ac8042cb8bb5af1ac1dda26,f03166ac11f0866596f9ea24c3d34d20,60d49ba3d611e129e56dca6b5021ecc6,1c40e4cb42404419d0b4906da53dd45b,703493f4417c30ed1e1856d3628945a4,96a2f98e7c341eef28c3c464908e07e9
_id,1,1,1,1,1,1,1,1,1,1



--------------------------------------------------


,Тип,Значение,Количество значений,Количество пропусков,Количество уникальных значений,Медиана,Среднее,Значения
age,int64,возраст,24712,0,77,38.000,40.030,nan


,31,32,33,36,30,34,35,37,38,29
age,1170,1138,1107,1086,1046,1043,1016,902,877,860


...

,5228.1,5099.1,5191.0,5195.8,5076.2,5017.5,4991.6,5008.7,4963.6,5023.5
nr.employed,9831,5138,4582,2171,973,653,482,393,380,102



--------------------------------------------------


,Тип,Значение,Количество значений,Количество пропусков,Количество уникальных значений,Медиана,Среднее,Значения
target,int64,целевая переменная,24712,0,2,0.000,0.113,nan


,0,1
target,21925,2787



--------------------------------------------------


### Подготовка данных

In [49]:
from sklearn.preprocessing import StandardScaler

# подготовленные данные
dict_month = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12,
}

dict_day_of_week = {
    'mon': 1, 'tue': 2, 'wed': 3, 'thu': 4, 'fri': 5, 'sat': 6, 'sun': 7,
}

# количество лет обучения
dict_education = {
    "illiterate": 0, "basic.4y": 4, "basic.6y": 6, "basic.9y": 9, "high.school": 11, "professional.course": 14, "university.degree": 16,
}



# заменяем '.' на ',' и переводим строку в число
def convert_to_float(df, column):
    return df[column].map(lambda x: x.replace(',', '.')).astype('float')

# заменяем 'unknown' -> Null
def convert_unknown_to_null(column, df):
    return df[column.name].map(lambda x: np.nan if x=='unknown' else x)

def convert_all_unknown_to_null(df):
    return df.apply(convert_unknown_to_null, args=(df,))

# нормализация (мин/макс)
def normalize(df, columns):
    for feature_name in columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        df[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return df


def scaling(df, columns):
    scale = StandardScaler()
    df[columns] = scale.fit_transform(df[columns].values)
    return df;    

# заменяем '.' на ',' и переводим строку в число
def convert_to_float(df, column):
    return df[column].map(lambda x: x.replace(',', '.')).astype('float')

#передаем все поля на проверку пустых значений
def delete_all_empty_values(df):        
    df.dropna(inplace=True)
    return df

#берем наиболее встречающееся значение и заполняем все пропуска им
def fill_empty_values(column, df):
    most_freq_value = df[column.name].mode()[0]
    return df[column.name].fillna(most_freq_value)

#передаем все поля на заполнение
def fill_all_empty_values(df):    
    return df.apply(fill_empty_values, args=(df,))


def preproc_data(df_input):
    #df_output = df_input.copy()
    ## Your Code Here
    
    df_output = (df_input.copy()
                 # Удаляем идентификатор
                .drop('_id', axis=1)
                 # 'unknown' -> Null                
                .pipe(convert_all_unknown_to_null)            
                 # Удаляем записи с пропусками
                #.pipe(delete_all_empty_values)
                 # Заполняем пропуски наиболее популярными значениями (в нашем случае только категориальные)
                .pipe(fill_all_empty_values)                  
                 # Фича "не контактировали"
                #.assign(not_contacted = lambda x: np.where(x['pdays']==999, 1, 0))
                 # Feature Interactions
                .assign(job_contact = lambda df: df['job'] + '_' + df['contact'],
                        job_education = lambda df: df['job'] + '_' + df['education'],
                        job_default = lambda df: df['job'] + '_' + df['default'],
                        job_housing = lambda df: df['job'] + '_' + df['housing'],
                        job_loan = lambda df: df['job'] + '_' + df['loan'],
                        job_marital = lambda df: df['job'] + '_' + df['marital'],
                        job_poutcome = lambda df: df['job'] + '_' + df['poutcome'],
                        contact_education = lambda df: df['contact'] + '_' + df['education'],
                        contact_default = lambda df: df['contact'] + '_' + df['default'],
                        contact_housing = lambda df: df['contact'] + '_' + df['housing'],
                        contact_loan = lambda df: df['contact'] + '_' + df['loan'],
                        contact_marital = lambda df: df['contact'] + '_' + df['marital'],
                        contact_poutcome = lambda df: df['contact'] + '_' + df['poutcome'],
                        education_default = lambda df: df['education'] + '_' + df['default'],
                        education_housing = lambda df: df['education'] + '_' + df['housing'],
                        education_loan = lambda df: df['education'] + '_' + df['loan'],
                        education_marital = lambda df: df['education'] + '_' + df['marital'],
                        education_poutcome = lambda df: df['education'] + '_' + df['poutcome'],
                        default_housing = lambda df: df['default'] + '_' + df['housing'],
                        default_loan = lambda df: df['default'] + '_' + df['loan'],
                        default_marital = lambda df: df['default'] + '_' + df['marital'],
                        default_poutcome = lambda df: df['default'] + '_' + df['poutcome'],
                        housing_loan = lambda df: df['housing'] + '_' + df['loan'],
                        housing_marital = lambda df: df['housing'] + '_' + df['marital'],
                        housing_poutcome = lambda df: df['housing'] + '_' + df['poutcome'],
                        loan_marital = lambda df: df['loan'] + '_' + df['marital'],
                        loan_poutcome = lambda df: df['loan'] + '_' + df['poutcome'],
                        marital_poutcome = lambda df: df['marital'] + '_' + df['poutcome'],  
                )
                 # Переводим в бинарный вид
                .assign(default = lambda x: np.where(x['default']=='yes', 1, 0),
                        housing = lambda x: np.where(x['housing']=='yes', 1, 0),
                        loan = lambda x: np.where(x['loan']=='yes', 1, 0),
                )  
                 # Нормализация 
                #.pipe(scaling, ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx',
                #     'cons.conf.idx', 'euribor3m', 'nr.employed'])
                .pipe(normalize, ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx',
                     'cons.conf.idx', 'euribor3m', 'nr.employed'])
                 # Кодировка категориальных признаков (one hot encoding)
                .pipe(pd.get_dummies, drop_first=True, columns=['job', 'marital', 'contact', 'poutcome'])
                 # Кодировка категориальных признаков (one hot encoding) - feature interactions
                .pipe(pd.get_dummies, drop_first=True, columns=['job_contact', 'job_education', 'job_default', 'job_housing', 'job_loan', 'job_marital', 'job_poutcome', 'contact_education', 'contact_default', 'contact_housing', 'contact_loan', 'contact_marital', 'contact_poutcome', 'education_default', 'education_housing', 'education_loan', 'education_marital', 'education_poutcome', 'default_housing', 'default_loan', 'default_marital', 'default_poutcome', 'housing_loan', 'housing_marital', 'housing_poutcome', 'loan_marital', 'loan_poutcome', 'marital_poutcome', ])                  
                 # Dictionary encoding
                .assign(month = lambda df: df['month'].map(lambda x: dict_month[x]),
                        day_of_week = lambda df: df['day_of_week'].map(lambda x: dict_day_of_week[x]),
                        education = lambda df: df['education'].map(lambda x: dict_education[x]),
                       )
                 # В качестве оптимизации удалим малозначимые колонки (ухудшило результаты)
                .drop(['education_loan_university.degree_yes','education_loan_high.school_yes','job_housing_technician_yes','contact_loan_cellular_yes','job_housing_technician_no','job_loan_admin._yes','job_education_blue-collar_basic.6y','education_default_professional.course_no','job_default_management_no','job_housing_services_no','job_poutcome_blue-collar_nonexistent','job_marital_blue-collar_single','job_poutcome_admin._success','job_student','job_education_retired_basic.4y','contact_education_cellular_basic.9y','job_loan_retired_no','education_marital_university.degree_divorced','education_housing_basic.9y_no','job_contact_admin._telephone','job_housing_services_yes','education_poutcome_professional.course_nonexistent','loan_poutcome_yes_success','loan_marital_yes_married','education_poutcome_basic.4y_nonexistent','job_education_technician_university.degree','job_housing_retired_yes','education_poutcome_basic.9y_nonexistent','marital_poutcome_divorced_nonexistent','job_housing_unemployed_no','job_marital_technician_married','loan_poutcome_yes_failure','job_poutcome_technician_nonexistent','housing_poutcome_no_success','job_unemployed','job_marital_management_married','job_poutcome_management_nonexistent','job_education_blue-collar_high.school','contact_education_telephone_high.school','education_marital_basic.6y_married','job_poutcome_retired_nonexistent','job_housing_entrepreneur_yes','job_management','job_education_admin._basic.9y','job_education_technician_professional.course','job_housing_unemployed_yes','education_loan_basic.6y_no','job_contact_self-employed_cellular','job_default_housemaid_no','job_education_student_basic.9y','education_loan_basic.4y_yes','job_poutcome_student_failure','job_marital_technician_divorced','job_services','job_housing_student_no','education_housing_basic.6y_no','job_marital_retired_divorced','education_default_basic.6y_no','job_loan_entrepreneur_no','job_default_self-employed_no','job_poutcome_self-employed_nonexistent','job_contact_management_cellular','job_poutcome_technician_failure','job_housing_self-employed_yes','job_housing_blue-collar_yes','job_poutcome_unemployed_nonexistent','job_housing_entrepreneur_no','education_marital_high.school_divorced','education_marital_basic.6y_single','job_contact_services_cellular','job_housemaid','job_loan_blue-collar_yes','job_marital_technician_single','job_default_services_no','contact_education_cellular_basic.6y','loan_marital_yes_single','job_education_services_high.school','job_loan_self-employed_yes','contact_loan_telephone_yes','job_contact_student_cellular','job_loan_retired_yes','job_education_student_high.school','job_marital_retired_married','job_education_unemployed_high.school','job_marital_entrepreneur_married','job_housing_self-employed_no','job_loan_unemployed_no','education_poutcome_basic.9y_success','marital_poutcome_single_success','education_poutcome_high.school_failure','job_loan_student_no','job_education_self-employed_university.degree','job_education_retired_university.degree','job_marital_management_single','job_marital_services_single','job_education_entrepreneur_professional.course','contact_education_telephone_university.degree','job_poutcome_housemaid_nonexistent','job_marital_self-employed_married','job_default_entrepreneur_no','job_marital_unemployed_married','job_poutcome_entrepreneur_nonexistent','contact_education_telephone_professional.course','job_contact_management_telephone','education_housing_basic.6y_yes','education_poutcome_professional.course_failure','job_housing_retired_no','job_poutcome_technician_success','job_education_student_university.degree','contact_marital_telephone_single','job_default_unemployed_no','job_marital_housemaid_married','job_contact_services_telephone','job_education_services_basic.9y','job_marital_entrepreneur_divorced','job_education_housemaid_basic.4y','job_marital_blue-collar_divorced','job_poutcome_student_success','job_housing_housemaid_no','job_education_technician_high.school','job_education_retired_professional.course','contact_poutcome_telephone_success','job_education_blue-collar_basic.9y','education_loan_professional.course_yes','job_marital_management_divorced','job_loan_housemaid_yes','job_marital_housemaid_single','contact_education_telephone_basic.4y','job_marital_unemployed_single','job_contact_unemployed_cellular','job_education_admin._basic.6y','job_education_entrepreneur_university.degree','job_entrepreneur','job_education_unemployed_university.degree','job_education_blue-collar_university.degree','education_poutcome_basic.9y_failure','job_loan_self-employed_no','job_contact_retired_telephone','job_marital_retired_single','job_poutcome_services_failure','job_education_unemployed_professional.course','job_loan_services_yes','job_education_services_university.degree','job_marital_self-employed_single','job_education_blue-collar_basic.4y','job_education_housemaid_university.degree','job_education_retired_basic.9y','job_loan_housemaid_no','job_contact_entrepreneur_telephone','job_marital_services_divorced','job_loan_unemployed_yes','job_contact_student_telephone','education_marital_basic.9y_single','job_education_retired_high.school','job_contact_technician_telephone','education_marital_basic.4y_single','job_contact_blue-collar_telephone','contact_education_telephone_basic.9y','job_poutcome_blue-collar_success','contact_poutcome_telephone_failure','job_poutcome_retired_success','job_loan_management_yes','contact_education_telephone_basic.6y','education_loan_basic.9y_yes','contact_marital_telephone_divorced','job_housing_housemaid_yes','education_marital_professional.course_divorced','education_marital_basic.6y_divorced','job_education_housemaid_high.school','education_poutcome_basic.6y_nonexistent','education_loan_basic.6y_yes','job_poutcome_housemaid_success','education_marital_basic.9y_divorced','job_education_self-employed_basic.4y','loan_marital_yes_divorced','job_poutcome_management_failure','job_education_management_high.school','job_education_blue-collar_professional.course','job_poutcome_services_nonexistent','job_education_entrepreneur_high.school','job_education_admin._professional.course','job_education_housemaid_professional.course','job_contact_self-employed_telephone','job_contact_unemployed_telephone','job_poutcome_unemployed_failure','job_marital_entrepreneur_single','job_education_management_professional.course','job_contact_housemaid_cellular','job_poutcome_blue-collar_failure','job_education_student_professional.course','job_education_unemployed_basic.4y','job_education_services_basic.4y','job_education_services_basic.6y','job_education_self-employed_basic.6y','job_education_management_basic.9y','job_contact_entrepreneur_cellular','job_education_self-employed_professional.course','job_education_technician_basic.9y','job_poutcome_entrepreneur_failure','job_poutcome_management_success','marital_poutcome_divorced_success','job_poutcome_services_success','job_poutcome_retired_failure','job_education_management_basic.6y','job_education_self-employed_basic.9y','job_contact_housemaid_telephone','job_marital_unemployed_divorced','job_education_technician_basic.6y','job_education_technician_basic.4y','job_marital_housemaid_divorced','job_poutcome_self-employed_failure','job_education_entrepreneur_basic.9y','job_poutcome_unemployed_success','job_education_student_basic.4y','job_poutcome_self-employed_success','education_poutcome_basic.6y_failure','job_poutcome_entrepreneur_success','job_education_housemaid_basic.6y','job_marital_self-employed_divorced','job_poutcome_housemaid_failure','job_education_retired_basic.6y','job_education_self-employed_high.school','job_marital_student_married','job_loan_entrepreneur_yes','job_education_unemployed_basic.6y','education_housing_illiterate_no','job_education_services_professional.course','job_education_student_basic.6y','job_education_housemaid_basic.9y','education_marital_illiterate_divorced','job_education_entrepreneur_basic.4y','job_education_entrepreneur_basic.6y','job_education_blue-collar_illiterate','default_marital_yes_married','default_poutcome_yes_failure','default_loan_yes_no','default_housing_yes_yes','default_housing_yes_no','contact_education_cellular_illiterate','education_poutcome_illiterate_success','default_poutcome_yes_nonexistent','education_poutcome_illiterate_nonexistent','contact_default_cellular_yes','education_poutcome_illiterate_failure','job_education_entrepreneur_illiterate','default','education_poutcome_basic.6y_success','education_default_illiterate_no','job_default_unemployed_yes','education_loan_illiterate_no','job_education_management_basic.4y','education_default_high.school_yes','job_education_housemaid_illiterate','education_housing_illiterate_yes','job_education_retired_illiterate','job_education_admin._illiterate','education_loan_illiterate_yes','job_default_technician_yes','job_marital_student_divorced','job_education_self-employed_illiterate','job_education_unemployed_basic.9y','contact_education_telephone_illiterate','education_default_professional.course_yes','education_marital_illiterate_single','education_marital_illiterate_married', 
                      'marital_poutcome_married_success', 'job_contact_blue-collar_cellular', 'contact_marital_cellular_single', 'job_default_student_no', 'housing_poutcome_yes_success', 'job_default_retired_no', 'default_loan_no_yes', 'job_housing_admin._yes', 'job_marital_blue-collar_married', 'job_housing_student_yes', 'contact_education_cellular_university.degree', 'education_poutcome_high.school_nonexistent', 'education_poutcome_university.degree_nonexistent', 'contact_marital_telephone_married', 'job_default_technician_no', 'job_loan_technician_no', 'education_housing_university.degree_yes', 'housing_marital_yes_single', 'loan_marital_no_married', 'job_technician', 'job_marital_student_single', 'job_default_blue-collar_no', 'education_loan_high.school_no', 'education_housing_basic.9y_yes', 'education_marital_high.school_married', 'job_retired', 'loan_marital_no_single', 'housing_marital_yes_married', 'job_contact_technician_cellular', 'housing', 'default_marital_no_single', 'housing_loan_yes_no', 'loan_poutcome_yes_nonexistent', 'job_education_admin._university.degree', 'job_loan_management_no', 'education_loan_university.degree_no', 'contact_poutcome_cellular_nonexistent', 'contact_education_cellular_professional.course', 'job_marital_admin._single', 'marital_poutcome_single_failure', 'job_poutcome_student_nonexistent', 'job_loan_blue-collar_no', 'contact_loan_telephone_no', 'education_housing_professional.course_yes', 'education_poutcome_university.degree_failure', 'marital_poutcome_single_nonexistent', 'education_poutcome_professional.course_success', 'loan', 'default_housing_no_yes', 'housing_poutcome_no_nonexistent', 'job_education_admin._high.school', 'marital_single', 'housing_marital_yes_divorced', 'marital_married', 'education_loan_professional.course_no', 'education_marital_professional.course_single', 'education_default_high.school_no', 'contact_housing_telephone_yes', 'contact_marital_cellular_married', 'education_housing_university.degree_no', 'job_education_management_university.degree', 'contact_housing_cellular_yes', 'default_marital_no_married', 'education_marital_university.degree_single', 'job_housing_management_yes', 'job_loan_services_no', 'education_housing_professional.course_no', 'marital_poutcome_married_nonexistent', 'default_poutcome_no_nonexistent', 'housing_marital_no_single', 'job_poutcome_admin._nonexistent', 'education_loan_basic.9y_no', 'education_marital_basic.9y_married', 'job_blue-collar', 'education_housing_high.school_yes', 'job_housing_management_no', 'job_marital_services_married', 'job_housing_blue-collar_no', 'education_marital_university.degree_married', 'job_marital_admin._married', 'education_housing_high.school_no', 'housing_loan_yes_yes', 'contact_education_cellular_high.school', 'education_housing_basic.4y_yes', 'housing_poutcome_yes_nonexistent', 'education_marital_high.school_single', 'job_loan_technician_yes', 'housing_marital_no_married', 'job_contact_retired_cellular', 'education_marital_professional.course_married', 'education_default_basic.9y_no', 'housing_poutcome_yes_failure', 'education_default_university.degree_no', 'education_marital_basic.4y_married', 'job_self-employed', 'housing_loan_no_yes', 'marital_poutcome_married_failure', 'job_loan_student_yes', 'education_poutcome_basic.4y_success', 'contact_housing_telephone_no', 'education_poutcome_high.school_success', 'contact_housing_telephone_no', 'education_poutcome_high.school_success', 'loan_poutcome_no_nonexistent', 'contact_poutcome_cellular_success', 'contact_poutcome_telephone_nonexistent', 'education_poutcome_university.degree_success', ], axis=1)
                #.drop(['age', 'default', 'housing', 'loan', 'job_admin.', 'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'marital_divorced', 'marital_single', 'education_basic.4y', 'education_basic.6y', 'education_basic.9y', 'education_high.school', 'education_illiterate', 'education_professional.course', 'education_university.degree', 'contact_telephone', 'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_sep', 'day_of_week_thu', 'day_of_week_tue', 'day_of_week_wed', 'poutcome_success',], axis=1)
    )        
    
    return df_output

In [50]:
df_train = df_train_origin.copy()
df_test = df_test_origin.copy()

df_train.loc[:, 'sample'] = 'train'
df_test.loc[:, 'sample'] = 'test'
df_ = df_test.append(df_train, sort=False).reset_index(drop=True)


df_preproc = df_.pipe(preproc_data)

df_train_preproc = df_preproc.query('sample == "train"').drop(['sample'], axis=1).reset_index(drop=True)
df_test_preproc = df_preproc.query('sample == "test"').drop(['sample'], axis=1).reset_index(drop=True)

print(df_train_preproc.shape)
print(df_test_preproc.shape)

df_train_preproc.head(10).T



(24712, 390)
(16476, 390)


,0,1,2,3,4,5,6,7,8,9
age,0.457,0.235,0.444,0.210,0.148,0.481,0.198,0.235,0.654,0.185
education,14.000,4.000,4.000,9.000,16.000,9.000,16.000,16.000,14.000,14.000
default,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
housing,0.000,0.000,1.000,1.000,1.000,0.000,1.000,1.000,0.000,1.000
loan,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000
month,8.000,5.000,8.000,5.000,8.000,6.000,5.000,11.000,10.000,5.000
day_of_week,2.000,1.000,1.000,3.000,5.000,4.000,1.000,3.000,4.000,4.000
duration,0.018,0.059,0.037,0.037,0.001,0.013,0.063,0.004,0.119,0.040
campaign,0.000,0.000,0.000,0.018,0.000,0.018,0.018,0.000,0.000,0.018
pdays,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.006,1.000


In [51]:
X_train, y_train = df_train_preproc.drop('target', axis=1), df_train_preproc['target']
X_test, y_test = df_test_preproc.drop('target', axis=1), []

## KNN

### Оптимизация точности

In [52]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV, StratifiedKFold, KFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

cv = StratifiedKFold(n_splits=10)

param_grid = {    
    'metric': ['manhattan'],
    'weights': ['distance'],
    'n_neighbors': [40], #np.linspace(35, 45, 11, dtype='int'),
}


model_KNN = KNeighborsClassifier()
model_KNN.fit(X_train,y_train)

grid_search_KNN = GridSearchCV(model_KNN, param_grid=param_grid, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=10)

grid_search_KNN.fit(X_train, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:  4.9min remaining: 11.5min
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  4.9min remaining:  4.9min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  5.0min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  6.7min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
       error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'metric': ['manhattan'], 'weights': ['distance'], 'n_neighbors': [40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=10)

In [54]:
print(grid_search_KNN.best_score_)
print(grid_search_KNN.best_params_)
#print(grid_search_RFC.grid_scores_)

0.7650776085623773
{'metric': 'manhattan', 'n_neighbors': 40, 'weights': 'distance'}


In [ ]:
'''
1) Как в RF
0.8865283269790721
{'metric': 'manhattan', 'n_neighbors': 40, 'weights': 'distance'}

2) Все фичи








'''

In [48]:
# находим веса
clf = KNeighborsClassifier(metric= 'manhattan', n_neighbors= 40, weights= 'distance')

n_feats = X_train.shape[1]
feature_importance_list = []

for i in range(n_feats):
    X_ = X_train[X_train.columns[i]].values.reshape(-1, 1)
    scores = cross_val_score(clf, X_, y_train)
    feature_importance_list.append([X_train.columns[i], scores.mean()])

feature_importance_list.sort(key=lambda x: x[1], reverse=True)
feature_importance_list

[['pdays', 0.8976206363380538],
 ['euribor3m', 0.8898105950379982],
 ['nr.employed', 0.8885562845754018],
 ['cons.price.idx', 0.8882729264107111],
 ['duration', 0.8876658896434922],
 ['emp.var.rate', 0.8873017186714348],
 ['education', 0.8872207830557173],
 ['day_of_week', 0.8872207830557173],
 ['poutcome_nonexistent', 0.8872207830557173],
 ['age', 0.88665430251837],
 ['month', 0.8866138396228451],
 ['cons.conf.idx', 0.8863710573373623],
 ['campaign', 0.7795894809597392],
 ['contact_telephone', 0.7377508469870921],
 ['contact_default_telephone_no', 0.7377508469870921],
 ['previous', 0.6510387980170952],
 ['poutcome_success', 0.6362303115984854],
 ['default_poutcome_no_success', 0.6362303115984854],
 ['loan_poutcome_no_success', 0.6346925348998491]]

In [172]:
model_RF2 = RandomForestClassifier(random_state=42, class_weight= None, criterion= 'entropy', max_depth= 12, max_features= 'auto', n_estimators= 220)

In [173]:
# среднее по фолдам
kf = KFold(n_splits=10, shuffle=True, random_state=42)

proba_list = []

for train_index, validation_index in kf.split(X_train):
    inner_X_train = X_train.loc[train_index, :]
    inner_y_train = y_train.loc[train_index]
    #inner_X_validation = X_train.loc[validation_index, :]
    #inner_y_validation = y_train.loc[validation_index, :]    
    
    model_RF2.fit(inner_X_train, inner_y_train)    
    Y_test_RFC_proba = model_RF2.predict_proba(X_test)
    
    proba_list.append(list(map(lambda x: x[1], Y_test_RFC_proba)))

temp_pd = pd.DataFrame(proba_list).T
temp_pd['mean'] = temp_pd.mean(axis=1)
result_RFC = df_test.loc[:,('_id',)]
#берем единички
result_RFC['target'] = temp_pd['mean']
result_RFC.to_csv('result_KNN_MEAN.csv', index=False)

In [176]:
model_RF2.fit(X_train,y_train)

Y_test_RFC = model_RF2.predict(X_test)
Y_test_RFC_proba = model_RF2.predict_proba(X_test)

result_RFC = df_test.loc[:,('_id',)]
#берем единички
result_RFC['target'] = list(map(lambda x: x[1], Y_test_RFC_proba))
result_RFC.to_csv('result_KNN_OPT.csv', index=False)